In [ ]:
"""
For Google Colab

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/repos/Epilepsy_Microglia
%pip install -q -r requirements.txt
"""

In [2]:
# project name
project = "Epilepsy_Microglia"
dataset = "thrupp"
gse_accession = "GSE153807"
link = "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE153807&format=file"

In [3]:
# environment setting
from src.env_utils import detect_env, get_paths
env = detect_env()
paths = get_paths(project)
print(env)
print(paths["base"])

code-server
/home/neuro_demo_research/data_from_drive/Epilepsy_Microglia


In [11]:
from pathlib import Path
import os
if(env == "colab"):
    download_dir = Path(f"/content/downloads/{dataset}")
elif(env == "code-server"):
    download_dir = Path(f"/home/neuro_demo_research/downloads/{dataset}")
else:
    download_dir = Path(f"~/neuro_demo_research/downloads/{dataset}").expanduser()

raw_dir = paths["raw"] / dataset

os.makedirs(download_dir, exist_ok=True)
os.makedirs(raw_dir, exist_ok=True)

In [12]:
from src.scanpy_utils.pipelines import download_tsv_csv

download_tsv_csv(
    filename=gse_accession,
    link=link,
    download_dir=download_dir,
    raw_dir=raw_dir,
    label=dataset,
    remove_original=True,
    is_tar=True,
)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 81.5M  100 81.5M    0     0  13.3M      0  0:00:06  0:00:06 --:--:-- 18.8M


Turning GSM4654473_Nuc-RM95-2.raw.tsv to csv
Turning GSM4654470_Nuc-RM77-1.raw.tsv to csv
Turning GSM4654468_Nuc-RM102-1.raw.tsv to csv
Turning GSM4654467_Nuc-RM101-2.raw.tsv to csv
Turning GSM4654472_Nuc-RM95-1.raw.tsv to csv
Turning GSM4654471_Nuc-RM77-2.raw.tsv to csv
Turning GSM4654469_Nuc-RM102-2.raw.tsv to csv
Writing thrupp_GSM4654473_Nuc-RM95-2.raw
Writing thrupp_GSM4654470_Nuc-RM77-1.raw
Writing thrupp_GSM4654468_Nuc-RM102-1.raw
Writing thrupp_GSM4654467_Nuc-RM101-2.raw
Writing thrupp_GSM4654472_Nuc-RM95-1.raw
Writing thrupp_GSM4654471_Nuc-RM77-2.raw
Writing thrupp_GSM4654469_Nuc-RM102-2.raw
READING: /home/neuro_demo_research/data_from_drive/Epilepsy_Microglia/raw/thrupp/thrupp_GSM4654473_Nuc-RM95-2.raw.h5ad
WRITING: /home/neuro_demo_research/data_from_drive/Epilepsy_Microglia/raw/thrupp/thrupp_GSM4654473_Nuc-RM95-2.fixed.h5ad
READING: /home/neuro_demo_research/data_from_drive/Epilepsy_Microglia/raw/thrupp/thrupp_GSM4654470_Nuc-RM77-1.raw.h5ad
WRITING: /home/neuro_demo_researc